# Inversión Fija Bruta
INEGI

In [4]:
# library 
Packages <- c("tidyverse","lubridate","ggplot2", "dplyr", "seasonal", "ggseas", "ggfortify", "forecast", "mFilter")
invisible(lapply(Packages, library, character.only=TRUE))

#set strings as factors to false 
options(stringsAsFactors=FALSE)


In [2]:
mytimeseries <- function(var, lab_var){
     # raw_data 
    message=FALSE
    rawdata <- read_csv("data/ifb.csv", col_types = cols() )
    rawdata <- data.frame(mutate(rawdata, time = make_date(year, mes) ))
    #head(rawdata,3)

    #selección de datos, 2010 en adelante
    wrk <- rawdata %>% 
        filter(time>=as.Date("2010-01-01"))
    wrk <- ts(as_tibble(wrk),frequency=12, start=c(2010,1), end=c(2019,8))
    
    raw_data <- wrk[,var]
    # Seasonal adjustment X13
    seas_adj <- seas(raw_data)
    #str(seas_adj$data)
    #summary(seas_adj$data)
    #head(seas_adj$data,3)
    seas_adj$model$arima
    arima <- seas_adj$model$arima$model
    #modelo <- paste(" Modelo ARIMA:",arima,sep="")
    modelo <- ""
    
    # HP filter 
    hp_data <- hpfilter(seas_adj$data[,"seasonaladj"])
    #summary(hp_data$trend)
    #str(hp_data$trend)

    #plot_data

    plot_data <- ts(
        mutate(as_tibble(seas_adj$data), 
               hp_trend = hp_data$trend, 
               rawdata = raw_data, 
               year = wrk[,"year"], 
               mes=wrk[,"mes"], 
               time = make_date(year,mes)
              ), frequency=12, start=c(2010,1), end=c(2019,8)
                    )
    
    csv_data <- subset(data.frame(plot_data), select=c("year","mes","rawdata","seasonaladj","trend","hp_trend"))
    
    csvpath <- paste("output/",var,sep="")
    csvpath <- paste(csvpath,".csv", sep="")
    write_csv(csv_data, path=csvpath)

    # graph   
    sub_lab <- "Índice base 2013 = 100"
    
    autoplot(plot_data[,c("seasonaladj", "trend", "hp_trend")], size=.8) + 
        labs( title = lab_var, 
             subtitle = sub_lab, 
             x = "", 
             y = "", 
             caption=paste("Fuente: Elaboración Secretaría de Economía con datos de INEGI.",modelo,sep="")
        ) + 
        scale_x_continuous(breaks=seq(from=2010, to=2019, by=1)) +
        theme_bw() + 
        theme(
            plot.title = element_text(hjust=0.5),
            plot.subtitle = element_text(hjust=0.5),
            legend.position="bottom", 
            plot.caption = element_text(hjust=0)
        ) + 
        scale_colour_manual(
            values=c("blue","red", "black"), 
            name="", 
            labels=c("Serie desestacionalizada", "Tendencia-ciclo", "Filtro Hodrick–Prescott")
        ) 

    figname <- paste("fig/",var,sep="")
    figname <- paste(figname,".png", sep="")

    ggsave(figname, width=18,height=12,units="cm")
    rm(rawdata, wrk, raw_data, seas_adj, arima, modelo, hp_data, plot_data, csv_data,sub_lab, figname)

    #return(var)
}



In [3]:
# generación de series 
mytimeseries("ifb", "Inversión Fija Bruta")
mytimeseries("ifb_c", "IFB, Construcción")
mytimeseries("ifb_c_res", "IFB, Construcción Residencial")
mytimeseries("ifb_c_nr", "IFB, Construcción No Residencial")
mytimeseries("ifb_mq", "IFB, Maquinaria y Equipo")
mytimeseries("ifb_mq_nal","IFB, Maquinaria y Equipo: Nacional" )
mytimeseries("ifb_mq_nal_tran", "IFB, Maquinaria y Equipo: Nacional, Eq.Transporte")
mytimeseries("ifb_mq_nal_mq", "IFB, Maquinaria y Equipo: Nacional, Maq. Eq.")
mytimeseries("ifb_mq_im", "IFB, Maquinaria y Equipo: Importado")
mytimeseries("ifb_mq_im_tran", "IFB, Maquinaria y Equipo: Importado, Eq.Transporte")
mytimeseries("ifb_mq_im_mq", "IFB, Maquinaria y Equipo: Nacional, Maq. Eq.")

## Inversión Fija Bruta al mes de agosto de 2019 

![](fig/ifb.png)

![](fig/ifb_c.png)

![](fig/ifb_c_res.png)

![](fig/ifb_c_nr.png)

![](fig/ifb_mq.png)

![](fig/ifb_mq_nal.png)

![](fig/ifb_mq_nal_mq.png)

![](fig/ifb_mq_nal_tran.png)

![](fig/ifb_mq_im.png)

![](fig/ifb_mq_im_mq.png)
